## Precarga de librerias y funciones

In [1]:
%%capture
%run "../../recurrentes.ipynb" 

In [2]:
%%capture
%run "../../funciones.ipynb"

#### Excel

In [3]:
# 2 - Cargar datos
# Opciones:
# a) dataset_clase_ternaria_l
# b) dataset_lags_clase_ternaria_l
# c) dataset_lags_deltas_y_clase_ternaria_l
# d) dataset_10_meses_l

df_train = pd.read_parquet(dataset_clase_ternaria_l)

In [4]:
# df_train = df_train.drop(columns=dicc_psi['dataset_lags_deltas_y_clase_ternaria'])
# df_train = df_train.drop(columns=dicc_psi['lista_light_gbm_feature_importance'])

## Formateo pre modelo

In [5]:
if 'T_Visa_normal' in df_train.columns:
    df_train['T_Visa_normal'] = df_train['T_Visa_normal'].astype(bool)
if 'T_Master_normal'in df_train.columns:
    df_train['T_Master_normal'] = df_train['T_Master_normal'].astype(bool)


In [6]:
# 3 - parametros para modelo
ganancia_acierto = 273000
costo_estimulo = 7000
mes_train = [202011, 202012, 202101, 202102, 202103, 202104]
mes_test = 202106

In [7]:
data = df_train
data['clase_peso'] = 1.0
data.loc[data['clase_ternaria'] == 'BAJA+2', 'clase_peso'] = 1.00002
data.loc[data['clase_ternaria'] == 'BAJA+1', 'clase_peso'] = 1.00001
data['clase_binaria'] = np.where(data['clase_ternaria']=='CONTINUA', 0, 1)
data['foto_mes'].unique()


df_train = data[data['foto_mes'].isin(mes_train)]
df_test = data[data['foto_mes']==mes_test]
clase_peso = df_train['clase_peso']
X_train = df_train.drop(['clase_ternaria', 'clase_binaria', 'clase_peso'], axis=1)
Y_train =df_train['clase_binaria']
X_test = df_test.drop(['clase_ternaria', 'clase_binaria', 'clase_peso'], axis=1)
Y_test =df_test['clase_binaria']
w_train = df_train['clase_peso']
w_test = df_test['clase_peso']

## Predicciones

In [8]:
# best_params = {'num_leaves': 1796,
#  'learning_rate': 0.049283676079631966,
#  'min_data_in_leaf': 9,
#  'feature_fraction': 0.44425115760554584,
#  'bagging_fraction': 0.40692971865213867}

In [9]:
best_params = {'num_leaves': 3545,
 'learning_rate': 0.07959540527561224,
 'min_data_in_leaf': 933,
 'feature_fraction': 0.6829932939284065,
 'bagging_fraction': 0.6251908881121132}
best_iter = 5

In [10]:
numeros_random = np.random.randint(0, 100000, 20)

In [11]:


train_data = lgb.Dataset(X_train,
                            label=Y_train,
                            weight=w_train)

In [12]:

best_iter = 1789

df_voting = pd.DataFrame()

for x in semillas:
    params = {
    'objective': 'binary',
    'boosting_type': 'gbdt',
    'first_metric_only': True,
    'boost_from_average': True,
    'feature_pre_filter': False,
    'max_bin': 31,
    'num_leaves': best_params['num_leaves'],
    'learning_rate': best_params['learning_rate'],
    'min_data_in_leaf': best_params['min_data_in_leaf'],
    'feature_fraction': best_params['feature_fraction'],
    'bagging_fraction': best_params['bagging_fraction'],
    'seed': x,
    'verbose': 0
}
        
            
    model = lgb.train(params,
                    train_data,
                    num_boost_round=best_iter)

    y_pred = model.predict(X_test)
    df_voting[f'prediccion_seed_{x}'] = y_pred

df_voting['prediccion'] = df_voting.mean(axis=1)
df_voting.index = X_test.index



[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

In [13]:
prediccion = pd.DataFrame({'numero_de_cliente': X_test['numero_de_cliente'], 'probabilidad': df_voting ['prediccion']}, index=X_test.index) 
prediccion.shape 

(164876, 2)

In [14]:
prediccion.isna().sum()

numero_de_cliente    0
probabilidad         0
dtype: int64

In [15]:
bajas = pd.DataFrame({'numero_de_cliente': X_test['numero_de_cliente'], 'clase_ternaria': df_test['clase_ternaria']}, index= X_test.index)
bajas.shape

(164876, 2)

In [16]:
bajas.isna().sum()

numero_de_cliente    0
clase_ternaria       0
dtype: int64

## Back Testing Bajas

In [17]:
prediccion.shape

(164876, 2)

In [18]:
merged_data = prediccion.merge(bajas, on='numero_de_cliente', how='inner')


In [19]:
merged_data.columns

Index(['numero_de_cliente', 'probabilidad', 'clase_ternaria'], dtype='object')

In [21]:
merged_data_sorted = merged_data.sort_values('probabilidad', ascending=False)


In [22]:

# Ordenar por probabilidad de mayor a menor

# Seleccionar top clientes
top_clients = merged_data_sorted.iloc[:12000].copy()  # Trabajar con una copia

# Convertir 'clase_ternaria' a variable binaria para los top clientes
top_clients.loc[:, 'bajas_reales'] = (top_clients['clase_ternaria'] == 'BAJA+2').astype(int)

top_clients.value_counts('clase_ternaria')

clase_ternaria
CONTINUA    10830
BAJA+1        603
BAJA+2        567
Name: count, dtype: int64

In [23]:
top_clients.value_counts('bajas_reales')

bajas_reales
0    11433
1      567
Name: count, dtype: int64

## Punto de Corte

In [38]:
%%capture
inicio = time.time()

resultados = []
cortes = range(5000, 20000, 1000)


pred_model_sorted = prediccion.sort_values('probabilidad', ascending=False)

model_name = 'LightGBM'

# Iteramos sobre cada corte
for corte in cortes:
    lista_ganancia_publico =[]
    lista_ganancia_privado = []
    for semilla in numeros_random: 
        ganancia_publico, ganancia_privado = calculoGanancia(bajas, pred_model_sorted, corte, semilla)        
    promedio_ganancia_publico = np.mean(lista_ganancia_publico)
    promedio_ganancia_privado = np.mean(lista_ganancia_privado)
        
    resultados.append({
    'Modelo': model_name,
    'Corte': corte,
    'Ganancia Público': ganancia_publico,
    'Ganancia Privado': ganancia_privado
    })

# Convertimos los resultados en un DataFrame
resultados = pd.DataFrame(resultados)

fin = time.time()
tiempo_ejecucion = fin - inicio

print(f"Tiempo de ejecución: {tiempo_ejecucion:.2f} segundos")

In [25]:
# Pivotamos el DataFrame 'resultados' para reorganizar las ganancias
resultados_pivot = resultados.pivot_table(
    index='Corte',
    columns='Modelo',
    values=['Ganancia Público', 'Ganancia Privado']
)

# Aplanamos las columnas para facilitar el acceso
resultados_pivot.columns = [f'{ganancia}_{modelo}' for ganancia, modelo in resultados_pivot.columns]

# Reordenamos las columnas alternando 'Público' y 'Privado' para cada modelo
# Ordenamos primero por el modelo, luego alternando entre 'Público' y 'Privado'
columnas_ordenadas = []
for modelo in resultados['Modelo'].unique():
    columnas_ordenadas.append(f'Ganancia Público_{modelo}')
    columnas_ordenadas.append(f'Ganancia Privado_{modelo}')

# Reorganizamos el DataFrame usando el nuevo orden de columnas
resultados_pivot = resultados_pivot[columnas_ordenadas]

# Convertimos el índice 'Corte' en una columna si prefieres tenerla como tal
resultados_pivot = resultados_pivot.reset_index()
resultados_pivot

,Corte,Ganancia Público_LightGBM,Ganancia Privado_LightGBM
0,5000,5.894000e+07,66340000.0
1,6000,6.871667e+07,70550000.0
2,7000,6.969667e+07,75730000.0
3,8000,6.951000e+07,76610000.0
4,9000,6.909000e+07,78390000.0
5,10000,7.270667e+07,78440000.0
6,11000,6.983667e+07,78470000.0
7,12000,6.708333e+07,78050000.0
8,13000,6.437667e+07,74010000.0
9,14000,6.185667e+07,73090000.0


In [27]:
resultados_pivot.shape

(15, 3)

In [37]:
maxima_ganancia_publico = resultados_pivot['Ganancia Público_LightGBM'].max()
maxima_ganancia_privado = resultados_pivot['Ganancia Privado_LightGBM'].max()


for indice, resultado in resultados_pivot.iterrows():
    if resultado['Ganancia Público_LightGBM'] == maxima_ganancia_publico:
        corte_maxima_ganancia_publico = indice  # Puedes ajustar según la lógica de "corte"
        ganancia_si_corte_maxima_ganancia_publico = (
            resultado['Ganancia Público_LightGBM'] + resultado['Ganancia Privado_LightGBM']
        )
        print(f'Ganancia si se hace corte por máxima ganancia público: {ganancia_si_corte_maxima_ganancia_publico}, el corte será {corte_maxima_ganancia_publico}')
    
    if resultado['Ganancia Privado_LightGBM'] == maxima_ganancia_privado:
        corte_maxima_ganancia_privado = indice  # Puedes ajustar según la lógica de "corte"
        ganancia_si_corte_maxima_ganancia_privado = (
            resultado['Ganancia Público_LightGBM'] + resultado['Ganancia Privado_LightGBM']
        )
        print(f'Ganancia si se hace corte por máxima ganancia privado: {ganancia_si_corte_maxima_ganancia_privado}, el corte será {corte_maxima_ganancia_privado}')
               
corte_publico_ganancia_total = resultados_pivot.loc[corte_maxima_ganancia_publico,'Ganancia Privado_LightGBM'] + resultados_pivot.loc[corte_maxima_ganancia_publico,'Ganancia Público_LightGBM']
corte_privado_ganancia_total = resultados_pivot.loc[corte_maxima_ganancia_privado,'Ganancia Privado_LightGBM'] + resultados_pivot.loc[corte_maxima_ganancia_privado,'Ganancia Público_LightGBM']
corte_maxima_ganancia_total = corte_maxima_ganancia_privado if corte_privado_ganancia_total > corte_publico_ganancia_total else corte_maxima_ganancia_publico
print (f'El corte recomendado por máxima ganancia total es {corte_maxima_ganancia_total}')

Ganancia si se hace corte por máxima ganancia público: 151146666.6666667, el corte será 5
Ganancia si se hace corte por máxima ganancia privado: 148306666.6666667, el corte será 6
El corte recomendado por máxima ganancia total es 5
